In [1]:
import pandas as pd
import requests
import json
import matplotlib
%matplotlib inline

# API from skyscanner
The objective of this part is to extract data from Skyscanner about the flights from
    A. Berlin TXL to Cancun CUN
    B. Berlin TXL to Rio de Janeiro GIG 
    C. Berlin TXL to Buenos Aires EZE
For 3 different dates:
    A. 21-12-2019 - 27-12-2019
    B. 18-01-2020 - 24-01-2020
    C. 22-02-2020 - 28-02-2020

In [2]:
# it is done manualy on RapiApi,
# session key for TXL - CUN on 21-27.12.2019: 
# 84811c8b-0fc8-4661-b5d7-4f6afa50aeab' in euros

# SAVING THE KEY AND THEN ACCESING THE INFO WITH THE KEY

In [3]:
# In order to have accurate search for flight informations, i have to access the 
# keypass, this is from the POST create session

# departure date format 2019-12-21 , return date 2019-12-27, final_destinaction CUN
def get_key_pass(departure_date, return_date, final_destination):
    
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/v1.0"

    payload = ('inboundDate='
               +return_date
               +'&children=0&infants=0&country=US&currency=EUR&locale=en-US&originPlace=TXL-sky&destinationPlace='
               +final_destination
               +'-sky&outboundDate='
               +departure_date+'&adults=1')
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a",
    'content-type': "application/x-www-form-urlencoded"
    }

    response = requests.request("POST", url, data=payload, headers=headers)

    return('/'.join(dict(response.headers)['Location'].split('/')[3:]))

In [4]:
# Now that I have the key pass for the search, I can use it for the search. This could be a function
# this part is from GET POLL SESSION RESULT

key_pass = get_key_pass('2019-12-21', '2019-12-27', 'CUN')
def request_response(keypass):
    base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/"
    url = base_url + keypass
    querystring = {"pageIndex":"0","pageSize":"10", "sortType": "price"} # already sorting by price

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a"
        }
    return(requests.request("GET", url, headers=headers, params=querystring))

In [6]:
response = request_response(key_pass)

In [22]:
def get_cheapest_flights(requested_response):
    
    iterinaries = pd.DataFrame(response.json()['Itineraries'])
    flight_agents = pd.DataFrame(response.json()['Agents'])

    legs_departure = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration', 'Stops']]
                         .rename(columns={'Id': 'OutboundLegId',
                             'Departure': 'Outbound_departure',
                             'Arrival': 'Outbound_arrival',
                             'Duration': 'Outbound_duration',
                             'Stops': 'Outbound_stops'}))

    legs_arrival = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration', 'Stops']]
                         .rename(columns={'Id': 'InboundLegId',
                             'Departure': 'Inbound_departure',
                             'Arrival': 'Inbound_arrival',
                             'Duration': 'Inbound_duration',
                             'Stops': 'Inbound_stops'}))
    df = (iterinaries
          .merge(legs_departure, on = 'OutboundLegId')
          .merge(legs_arrival, on = 'InboundLegId')
          .drop(columns=['OutboundLegId', 'InboundLegId', 'BookingDetailsLink'])
          )

    df['Agents'] = [i[0]['Agents'][0] for i in df['PricingOptions']]
    df['Price'] = [i[0]['Price'] for i in df['PricingOptions']]
    df['Link']  = [i[0]['DeeplinkUrl'] for i in df['PricingOptions']]

    df = df.drop(columns='PricingOptions')

    df['Outbound_stops'] = [len(i) for i in df['Outbound_stops']]
    df['Inbound_stops'] = [len(i) for i in df['Inbound_stops']]

    flight_agents = flight_agents[['Id', 'Name']]

    df = df.rename(columns={'Agents': 'Id'})
    df = df.merge(flight_agents, on = 'Id')
    
    return(df.sort_values('Price')[:3])

In [23]:
df = get_cheapest_flights(response)
df

,Outbound_departure,Outbound_arrival,Outbound_duration,Outbound_stops,Inbound_departure,Inbound_arrival,Inbound_duration,Inbound_stops,Id,Price,Link,Name
9,2019-12-21T14:45:00,2019-12-22T10:45:00,1560,3,2019-12-27T11:45:00,2019-12-28T07:55:00,850,1,4132306,1351.54,http://partners.api.skyscanner.net/apiservices...,United
0,2019-12-21T07:45:00,2019-12-21T18:35:00,1010,2,2019-12-27T06:45:00,2019-12-28T08:10:00,1165,2,4499219,1580.54,http://partners.api.skyscanner.net/apiservices...,Expedia
1,2019-12-21T07:15:00,2019-12-21T21:37:00,1222,2,2019-12-27T13:41:00,2019-12-28T19:05:00,1404,2,4499219,1845.32,http://partners.api.skyscanner.net/apiservices...,Expedia
